<a href="https://colab.research.google.com/github/RedietNegash/Neural_Network_Simulation/blob/main/L_Layer_deep_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math


-> Initalize weights and bias for the L-layers

->L is for number of layers in the network

In [2]:
def initialize_parameters_deep(layer_dims):

    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        parameters['W' + str(l)] = [[random.uniform(-0.01, 0.01) for _ in range(layer_dims[l-1])] for _ in range(layer_dims[l])]
        parameters['b' + str(l)] = [0] * layer_dims[l]

    return parameters

#Forward Propagation

Linear combination-> Zj=∑i(Ai⋅Wji)+bj

RLU-> A1=ReLU(Z1)

Sigmoid-> Z1=W1⋅X+b1



Apply the linear combination of inputs A and weights W, add bias b for each output

In [28]:

def linear_forward(A, W, b):
    Z = [sum(A[i] * W[j][i] for i in range(len(A))) + b[j] for j in range(len(b))]
    return Z


Apply ReLU function to each element in Z, replacing negative values with 0

In [17]:
def relu(Z):
    return [max(0, z) for z in Z]

Apply the sigmoid function to each element in Z to map a real number to a value between 0 and 1

In [18]:
def sigmoid(Z):
    return [1 / (1 + math.exp(-z)) for z in Z]

-> For each layer except the output layer   transformation followed by ReLU activation

-> For the final layer apply linear transformation followed by sigmoid activation

In [19]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2


    for l in range(1, L):
        A_prev = A
        W = parameters['W' + str(l)]
        b = parameters['b' + str(l)]
        Z = linear_forward(A_prev, W, b)
        A = relu(Z)
        caches.append((A_prev, W, b, Z))

    WL = parameters['W' + str(L)]
    bL = parameters['b' + str(L)]
    ZL = linear_forward(A, WL, bL)
    AL = sigmoid(ZL)
    caches.append((A, WL, bL, ZL))

    return AL, caches

Apply the cost function

In [20]:
def compute_cost(AL, Y):
    m = len(Y)
    cost = -sum(Y[i] * math.log(AL[i]) + (1 - Y[i]) * math.log(1 - AL[i]) for i in range(m)) / m
    return cost

Pass the gradient back only for positive ReLU inputs otherwise set it to 0


In [21]:
def relu_backward(dA, Z):
    return [dA[i] if Z[i] > 0 else 0 for i in range(len(Z))]

Calculate the gradient of the loss function with respect to the input of the sigmoid activation function.

In [22]:
def sigmoid_backward(dA, Z):
    s = sigmoid(Z)
    return [dA[i] * s[i] * (1 - s[i]) for i in range(len(s))]

calcualte gradients for weights, biases, and previous layer's activations


In [23]:
def linear_backward(dZ, A_prev, W):
    m = len(A_prev)
    dW = [[sum(dZ[j] * A_prev[i][k] for k in range(m)) / m for i in range(len(A_prev))] for j in range(len(W))]
    db = [sum(dZ[j] for j in range(len(dZ))) / len(dZ) for j in range(len(dZ))]
    dA_prev = [sum(W[i][j] * dZ[j] for j in range(len(dZ))) for i in range(len(A_prev))]
    return dA_prev, dW, db


calculate the gradients for all layers during the backpropagation

In [24]:
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = len(Y)

    dAL = [(AL[i] - Y[i]) / (AL[i] * (1 - AL[i])) for i in range(len(Y))]
    current_cache = caches[L-1]
    A_prev, W, b, Z = current_cache
    dZL = sigmoid_backward(dAL, Z)
    dA_prev, dWL, dbL = linear_backward(dZL, A_prev, W)
    grads["dW" + str(L)] = dWL
    grads["db" + str(L)] = dbL

    for l in reversed(range(L-1)):
        current_cache = caches[l]
        A_prev, W, b, Z = current_cache
        dA = dA_prev
        dZ = relu_backward(dA, Z)
        dA_prev, dW, db = linear_backward(dZ, A_prev, W)
        grads["dW" + str(l+1)] = dW
        grads["db" + str(l+1)] = db
        dA_prev = dA_prev

    return grads

Update model parameters using gradient descent with the given learning rate.


In [25]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2
    for l in range(1, L + 1):
        parameters['W' + str(l)] = [[parameters['W' + str(l)][i][j] - learning_rate * grads['dW' + str(l)][i][j] for j in range(len(parameters['W' + str(l)][0]))] for i in range(len(parameters['W' + str(l)]))]
        parameters['b' + str(l)] = [parameters['b' + str(l)][i] - learning_rate * grads['db' + str(l)][i] for i in range(len(parameters['b' + str(l)]))]

    return parameters


Train a deep neural network by iterating through forward and backward propagation.



In [63]:
def L_layer_model(X, Y, layers_dims, learning_rate=0.0075, num_iterations=3000, print_cost=False):
    costs = []

    parameters = initialize_parameters_deep(layers_dims)

    for i in range(0, num_iterations):
        AL, caches = L_model_forward(X, parameters)
        cost = compute_cost(AL, Y)
        grads = L_model_backward(AL, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate)

        if print_cost and i % 100 == 0:
            print("Cost after iteration {}: {}".format(i, cost))
            costs.append(cost)

    return parameters

Test